## fill_antibody_table-dev

In this notebook, I populate the AntibodyInvenstory() table from the existing google spreadsheet here: https://docs.google.com/spreadsheets/d/1U_8icZOKXRR5s6lM8OnfnwX6V95E9upJbPbmwbZPvfk/edit#gid=524028263

In [57]:
import pickle
import os, sys
from datetime import datetime
import pandas as pd
import numpy as np
import datajoint as dj

In [44]:
history_csv_file = './antibodies_history.csv'
overview_csv_file = './antibodies_cheat_sheet.csv'

## Connect to the db

In [58]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.config['database.user'] = os.environ.get('DJ_DB_USER')
dj.config['database.password'] = os.environ.get('DJ_DB_PASS')
dj.conn()

DataJoint connection (connected) ahoag@datajoint00.pni.princeton.edu:3306

In [59]:
# set up object for light sheet schema
db_lightsheet = dj.create_virtual_module('ahoag_lightsheet_demo','ahoag_lightsheet_demo')
# db_lightsheet = dj.create_virtual_module('u19lightserv_lightsheet','u19lightserv_lightsheet')

## Antibody history

In [60]:
# Here are the columns
db_lightsheet.AntibodyHistory()

date The date this antibody combo was attempted,brief_descriptor,animal_model,primary_antibody,secondary_antibody,primary_concentration,secondary_concentration,primary_order_info,secondary_order_info,notes,username,request_name


In [61]:
df_history = pd.read_csv(history_csv_file,)
df_history = df_history.fillna('')
df_history.head(5)

,Date,Brief exp description,Animal model,Primary antibody,Primary order info,Primary concentration used,Secondary antibody,Secondary order info,Secondary concentration used,Comments
0,5/30/2016,cFos,B6/C57,Donkey antiRb AF568,,1 : 200,Donkey antiChicken,,1 : 500,
1,3/25/2016,MDThal,B6/C57,?,,,Donkey antiRabbit 568,,1 : 400,
2,3/25/2016,ACC,B6/C57,?,,,Donkey antiRabbit 568,,1 : 400,
3,3/25/2016,Datcre-VTA PRV,B6/C57,antiPRV,,1 : 500,Donkey antiChicken 568,,1 : 400,
4,3/25/2016,Datcre-VTA PRV,B6/C57,antiPRV,,1 : 500,Donkey antiChicken 568,,1 : 400,


In [62]:
df_history.columns

Index(['Date', 'Brief exp description', 'Animal model', 'Primary antibody',
       'Primary order info', 'Primary concentration used',
       'Secondary antibody', 'Secondary order info',
       'Secondary concentration used', 'Comments'],
      dtype='object')

In [63]:
history_insert_list = []
for index, row in df_history.iterrows():
    history_insert_dict = {}
    history_insert_dict['date'] = datetime.strptime(row['Date'],'%m/%d/%Y').strftime('%Y-%m-%d')
    history_insert_dict['brief_descriptor'] = row['Brief exp description']
    history_insert_dict['animal_model'] = row['Animal model']
    history_insert_dict['primary_antibody'] = row['Primary antibody']
    history_insert_dict['secondary_antibody'] = row['Secondary antibody']
    history_insert_dict['primary_concentration'] = row['Primary concentration used']
    history_insert_dict['secondary_concentration'] = row['Secondary concentration used']
    history_insert_dict['primary_order_info'] = row['Primary order info']
    history_insert_dict['secondary_order_info'] = row['Secondary order info']
    history_insert_dict['notes'] = row['Comments']
    history_insert_list.append(history_insert_dict)
db_lightsheet.AntibodyHistory.insert(history_insert_list,skip_duplicates=True)

## Antibody overview

In [46]:
# Here are the columns
db_lightsheet.AntibodyOverview()

brief_descriptor,animal_model,primary_antibody,secondary_antibody,primary_concentration,secondary_concentration,primary_order_info,secondary_order_info,notes


In [45]:
df_overview = pd.read_csv(overview_csv_file,)
df_overview = df_overview.fillna('')
df_overview.head(5)

,Brief exp description,Animal model,Primary antibody,Primary order info,Primary concentration used,Secondary antibody,Secondary order info,Secondary concentration used,Notes
0,anterotrap,B6/C57,chicken antiGFP,Aves,1 : 1500,Donkey antiChicken 647,,1 : 500,dilute powder in 400 ul water
1,cFos,B6/C57,Synaptic Systems rabbit anti cFos,SySy,1 : 1000,Donkey antiRabbit 647,,1 : 200,dilute powder in 50 ul water
2,DREADDs for mcherry cerebellar expression,B6/C57,rabbit antiRFP,rockland,1 : 1000,Donkey antiRabbit 647,,1 : 450,
3,EAAT4,EAAT4,chicken antiGFP,Aves,1 : 500,Donkey antiChicken 647,,1 : 750,dilute powder in 400 ul water
4,HSV H129,B6/C57,rabbit antiHSV,DAKO,1 : 350,Donkey antiRabbit 647,,1 : 250,


In [47]:
overview_insert_list = []
for index, row in df_overview.iterrows():
    overview_insert_dict = {}
    overview_insert_dict['brief_descriptor'] = row['Brief exp description']
    overview_insert_dict['animal_model'] = row['Animal model']
    overview_insert_dict['primary_antibody'] = row['Primary antibody']
    overview_insert_dict['secondary_antibody'] = row['Secondary antibody']
    overview_insert_dict['primary_concentration'] = row['Primary concentration used']
    overview_insert_dict['secondary_concentration'] = row['Secondary concentration used']
    overview_insert_dict['primary_order_info'] = row['Primary order info']
    overview_insert_dict['secondary_order_info'] = row['Secondary order info']
    overview_insert_dict['notes'] = row['Notes']
    overview_insert_list.append(overview_insert_dict)
db_lightsheet.AntibodyOverview.insert(overview_insert_list,skip_duplicates=True)

# Ingest requests into antibody history table

In [64]:
request_contents = db_lightsheet.Request()
clearing_batch_contents = request_contents.ClearingBatch()
combined_contents = request_contents*clearing_batch_contents

In [65]:
history_insert_list = []
for d in combined_contents:
    antibody1 = d['antibody1']
    antibody2 = d['antibody2']
    if not (antibody1 or antibody2):
        continue
    history_insert_dict = {}
    history_insert_dict['date'] = d['date_submitted']
    history_insert_dict['username'] = d['username']
    history_insert_dict['request_name'] = d['request_name']
    history_insert_dict['brief_descriptor'] = d['description'][0:128]
    history_insert_dict['animal_model'] = d['species']
    history_insert_dict['primary_antibody'] = d['antibody1']
    history_insert_dict['secondary_antibody'] = d['antibody2']
    history_insert_dict['primary_order_info'] = d['antibody1_lot']
    history_insert_dict['secondary_order_info'] = d['antibody2_lot']
    history_insert_list.append(history_insert_dict)
#     print(d['antibody1'],"   ...   ",d['antibody2'])
#     print(history_insert_dict)
db_lightsheet.AntibodyHistory.insert(history_insert_list,replace=True)